## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin7_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc7/o3/o3_bincore7.csv", index_col=0)
print(bin7_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin7_3.reset_index(inplace=True, drop=True)

print('shape -', bin7_3.shape)
print('reset_index 완료')
print('input data shape')
bin7_3.head()

(37418056, 2)
shape - (37418056, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,54,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin7_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin7_3['label'].value_counts())

256
0    37381720
1       36336
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin7_3[bin7_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 192 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin7_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin7_3', len(ls_idx_bin))

# loc 로 수정필요
bin7_3_Ngram = bin7_3.loc[ls_idx_bin,:].copy()

6976512
나머지 0
최종 길이 6976512
bin7_3 6976512


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin7_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin7_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin7_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin7_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

36336.0
0 26718917
1000 4046714
2000 22729740
3000 14388832
4000 36544054
5000 16893513
6000 3999871
7000 25129209
8000 3510257
9000 20216361
10000 18615749
11000 29490846
12000 26138844
13000 29876184
14000 34762696
15000 12194470
16000 16688446
17000 3001362
18000 25215616
19000 9430938
20000 27921165
21000 2237101
22000 36088642
23000 31740449
24000 13776509
25000 9230298
26000 23625479
27000 24885352
28000 32713001
29000 7456128
30000 27325524
31000 25601929
32000 33460371
33000 12754756
34000 25118239
35000 33789537
36000 2620848
완료
36336


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin7_3['label'].value_counts()[1])

36336
36336


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin7_3_Ngram])
final.shape

(13953024, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc7_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc7_3_onehot_Ngram = pd.concat([final['label'], bc7_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc7_3_onehot_Ngram.shape)

원핫인코딩완료
(13953024, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc7_3 = bc7_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc7_3 = bc7_3_onehot_Ngram['label'].to_numpy()
print(x_bc7_3.shape, x_bc7_3.shape)

x_bc7_3 = x_bc7_3.reshape(-1, right_idx, x_bc7_3.shape[1])
y_bc7_3 = y_bc7_3.reshape(-1, right_idx, 1)

print(x_bc7_3.shape, y_bc7_3.shape)

(13953024, 256) (13953024, 256)
(72672, 192, 256) (72672, 192, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc7_3.shape[0])

x_bc7_3 = x_bc7_3[p]
y_bc7_3 = y_bc7_3[p]

print(x_bc7_3.shape, y_bc7_3.shape)

(72672, 192, 256) (72672, 192, 1)


In [10]:
np.save('x_bc7_3', x_bc7_3)
np.save('y_bc7_3', y_bc7_3)

## (7) 모델

In [11]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(288, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [12]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bc7_3,
           y_bc7_3,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)

model1.save('gcc7_bin_core_s192_h288_o3.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192, 256)]        0         
_________________________________________________________________
bidirectional (Bidirectional (None, 192, 576)          1255680   
_________________________________________________________________
time_distributed (TimeDistri (None, 192, 1)            577       
Total params: 1,256,257
Trainable params: 1,256,257
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 50870 samples, validate on 21802 samples
Epoch 1/500
50870/50870 [==============================] - 73s 1ms/sample - loss: 0.0101 - accuracy: 0.9968 - val_loss: 0.0045 - val_accuracy: 0.9985
Epoch 2/500
50870/50870 [==============================] - 65s 1ms/sample - loss: 0.0027 - accuracy: 0.9991 - val_loss: 0.0018 - val_accuracy: 0.9994
Epoc

걸린시간: 1713.8816578388214
save 완료
